In [22]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import PrintfTickFormatter
import statistics
import random
import numpy as np
import scipy as sp
import bokeh
import math
import re

NUM_HOSTS = 2
HOSTS = ('Mac', 'other machine')
date_pattern = '%Y-%m-%dT%H-%M-%S'
date_pattern_new = '%Y-%m-%dT%H-%M-%S-%f'
things_to_extract_fathom = {'first_hop':
                        {'index': 1,
                         'name': 'first hop',
                         'line_dash': 'dashed',
                         'marker': 'circle',
                         'color': 'green'
                        }, 
                     'destination':
                         {'index': 0,
                          'name': 'mlab',
                          'line_dash': 'solid',
                          'marker' : 'triangle', 
                          'color': 'red'
                         },
                     'other_machine':
                         {'index': 2,
                          'name': 'other machine',
                          'line_dash': 'dotted',
                          'marker': 'square',
                          'color': 'blue'
                        }               
                    }
things_to_extract_webrtc = {'chrome':
                        {'index': 0,
                         'name': 'chrome',
                         'line_dash': 'dashed',
                         'marker': 'circle',
                         'color': 'green'
                        }, 
                     'firefox':
                         {'index': 1,
                          'name': 'firefox',
                          'line_dash': 'solid',
                          'marker' : 'triangle', 
                          'color': 'red'
                         },
                     'ping':
                         {'index': 2,
                          'name': 'system ping',
                          'line_dash': 'dotted',
                          'marker': 'square',
                          'color': 'blue'
                        }
                    }
things_to_extract_time = {'chrome':
                        {'index': 0,
                         'name': 'chrome',
                         'line_dash': 'dashed',
                         'marker': 'circle',
                         'color': 'green'
                        }, 
                     'firefox':
                         {'index': 1,
                          'name': 'firefox',
                          'line_dash': 'solid',
                          'marker' : 'triangle', 
                          'color': 'red'
                         },
                     'safari':
                         {'index': 2,
                          'name': 'safari',
                          'line_dash': 'dotted',
                          'marker': 'square',
                          'color': 'blue'
                        }
                    }
pings_per_run = 2
color_per_host = ['blue', 'red']

markers_by_type = {'ping': 'circle', 'result': 'square'}
line_style_by_type = {'ping': 'solid', 'result': 'dashed'}
alpha_color = 0.3
line_width = 0.1
all_css_colors = ["AliceBlue","AntiqueWhite","Aqua","Aquamarine","Azure","Beige","Bisque","Black","BlanchedAlmond","Blue","BlueViolet","Brown","BurlyWood","CadetBlue","Chartreuse","Chocolate","Coral","CornflowerBlue","Cornsilk","Crimson","Cyan","DarkBlue","DarkCyan","DarkGoldenRod","DarkGray","DarkGreen","DarkKhaki","DarkMagenta","DarkOliveGreen","Darkorange","DarkOrchid","DarkRed","DarkSalmon","DarkSeaGreen","DarkSlateBlue","DarkSlateGray","DarkTurquoise","DarkViolet","DeepPink","DeepSkyBlue","DimGray","DodgerBlue","FireBrick","FloralWhite","ForestGreen","Fuchsia","Gainsboro","GhostWhite","Gold","GoldenRod","Gray","Green","GreenYellow","HoneyDew","HotPink","IndianRed","Indigo","Ivory","Khaki","Lavender","LavenderBlush","LawnGreen","LemonChiffon","LightBlue","LightCoral","LightCyan","LightGoldenRodYellow","LightGray","LightGreen","LightPink","LightSalmon","LightSeaGreen","LightSkyBlue","LightSlateGray","LightSteelBlue","LightYellow","Lime","LimeGreen","Linen","Magenta","Maroon","MediumAquaMarine","MediumBlue","MediumOrchid","MediumPurple","MediumSeaGreen","MediumSlateBlue","MediumSpringGreen","MediumTurquoise","MediumVioletRed","MidnightBlue","MintCream","MistyRose","Moccasin","NavajoWhite","Navy","OldLace","Olive","OliveDrab","Orange","OrangeRed","Orchid","PaleGoldenRod","PaleGreen","PaleTurquoise","PaleVioletRed","PapayaWhip","PeachPuff","Peru","Pink","Plum","PowderBlue","Purple","Red","RosyBrown","RoyalBlue","SaddleBrown","Salmon","SandyBrown","SeaGreen","SeaShell","Sienna","Silver","SkyBlue","SlateBlue","SlateGray","Snow","SpringGreen","SteelBlue","Tan","Teal","Thistle","Tomato","Turquoise","Violet","Wheat","White","WhiteSmoke","Yellow","YellowGreen"]
shuffled_colors = list(all_css_colors)
random.shuffle(shuffled_colors)

# Load all datasets

In [28]:
# %debug
import os
import io
import json
import datetime
import statistics
import sys
from functools import reduce

## WebRTC overnight at home one machine
time_afternoon = (datetime.datetime.strptime('2016-04-29T17-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-29T19-00-00', date_pattern))

full_set = set(map(lambda item: str(item), range(pings_per_run)))
property_to_color = {}
measured_things = None

timestamp_start_time = None
highres_timestamp_start_time = None

def extract_data(start_date=None, end_date=None, mode='fathom', num_hosts=NUM_HOSTS):
    global timestamp_start_time
    global highres_timestamp_start_time
    if isinstance(start_date, list) or isinstance(start_date, set) or isinstance(start_date, tuple):
        start_date, end_date = start_date
    d = 'collected_data_time'

    extracted_data = []
    file_names = []
    files = [f for f in os.listdir(d) if os.path.isfile(os.path.join(d,f))]
#     start_timestamp = None
    for filename in sorted(files):
        if not filename.startswith('file'):
#             print("Skipped "+filename+" because it's some OS file")
            continue

        date_string = filename.split('_')[1].split('.')[0]
        file_date = datetime.datetime.strptime(date_string, date_pattern_new)
    
        if start_date and file_date < start_date:
            continue
        if end_date and file_date > end_date:
            break
    
        current_file = io.open(os.path.join(d,filename))
        content = current_file.read()

#         print("Processing "+filename)
        
        as_list = json.loads(content)
        as_list['datetime'] = file_date
        if not timestamp_start_time:
            timestamp_start_time = as_list['measuredByNode']['inaccurateTime']/1000
        if not highres_timestamp_start_time:
            highres_timestamp_start_time = as_list['measuredByNode']['measuredTime']/1000

        current_file.close()
        file_names.append(filename)
        extracted_data.append(as_list)
#     print(dictionaries)
#     print(extracted_data)
    return file_names, extracted_data


# Plot

In [29]:
# %debug
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import PrintfTickFormatter, HoverTool
output_notebook()
# output_file('plots.html')

def get_distribution(things):
    distribution = {}
    for thing in things:
        if not thing in distribution:
            distribution[thing] = 0
        distribution[thing] += 1
    return distribution

def get_cumulative_distribution(dist):
    cumulative_distribution = {}
    value_from_previous = 0
    for thing in sorted(dist.keys()):
        cumulative_distribution[thing] = value_from_previous + dist[thing]
        value_from_previous = cumulative_distribution[thing]
    return cumulative_distribution

def flatten(l):
    return [item for sublist in l for item in sublist]
    
plot_width = 1000
plot_height = 650
# NUM_HOSTS = 1
def plot_data(dataset, plot_type='fathom', things=[], thing_regex='', num_hosts=NUM_HOSTS):
    
    file_names, extracted_data = extract_data(dataset)
    
    things_to_extract = ['inaccurateTime', 'measuredTime', 'osTimestamp', 'osTimestampInaccurate']
    x = {}
    y = {}

    p = figure(tools="pan,box_zoom,reset,save,wheel_zoom", x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height)

    p.add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)"),("legend", "@desc")]))
    p.xaxis.axis_label="time"
    p.yaxis.axis_label="RTT"
    p.yaxis[0].formatter = PrintfTickFormatter(format="%.3f ms")

    for thing in things_to_extract:
        if thing_regex and not thing_regex.search(thing):
            continue
        if not thing in x:
            x[thing] = []
        if not thing in y:
            y[thing] = []

        y_values = [item['measuredByNode'][thing] for item in extracted_data]
        if thing=='inaccurateTime' or thing=='measuredTime':
            y_values = [item/1000 for item in y_values]
        
        if thing=='measuredTime':
            y_values = [item-highres_timestamp_start_time for item in y_values]
        else:
            y_values = [item-timestamp_start_time for item in y_values]
        
        y_values = [item-timestamp_start_time for item in y_values]
        y[thing] = y_values
        x_values = [item['datetime'] for item in extracted_data]
        x[thing] = x_values
        
    for thing in things_to_extract:
#         import pdb; pdb.set_trace()
        p.scatter('x', 'y', fill_alpha=alpha_color, color='black', marker='square', line_color=None, source=ColumnDataSource(data=dict(x=x[thing],y=y[thing],desc=[thing]*len(x[thing]))))

    show(p)
    
# plot_data(extracted_data_good, plot_type='all', thing_regex=re.compile('ajax.*172'))
# plot_data(good_all_measurements_overnight_chrome, plot_type='all', thing_regex=re.compile('^((?!WS).)*$'))
# plot_data(good_all_measurements_overnight_chrome_load, plot_type='all', thing_regex=re.compile('^((?!WS).)*$'))
plot_data(time_afternoon)
# plot_data(extracted_data_good, plot_type='all')

Loading BokehJS ...